In [1]:
!pip3 install -r requirements.txt
!pip3 install flask

Defaulting to user installation because normal site-packages is not writeable
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.2 requires click>=8.0, but you have click 7.1.2 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit

In [2]:


import cv2
from glob import glob
from pathlib import Path

from model import Model
from utils import plot

/var/data/python/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
model = Model()

# Read video 
### and save predictions for each frame as another video

In [4]:
def process_stream(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    prediction = model.predict(frame)
    label = prediction['label']
    conf = prediction['confidence']
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return label, conf, frame

In [5]:
from flask import Flask, render_template, Response, current_app, redirect, url_for, request
import flask
import socket
import cv2, time


        
app = Flask('hello')


app_context = app.app_context()


notifications = {"0":"Unknown"}

def gen_frames_camera(camera_num):  
    camera = cv2.VideoCapture(camera_num)  # CAP_DSHOW because of https://answers.opencv.org/question/234933/opencv-440modulesvideoiosrccap_msmfcpp-682-cvcapture_msmfinitstream-failed-to-set-mediatype-stream-0-640x480-30-mfvideoformat_rgb24unsupported-media/
    while True:
        success, frame = camera.read()
        if not success:
            break
        else:
            label, conf, frame = process_stream(frame)
            print(label, conf)
            if (label == "fight on a street") | (label == "street violence") | (label == "violence in office"):
                notifications["0"] = label
                


            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\nContent-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


def gen_frames_drone():
    tello_video = cv2.VideoCapture("udp://@0.0.0.0:11111")
    while True:
        try:
            ret, frame = tello_video.read()

            if ret:
                ret, buffer = cv2.imencode('.jpg', frame)
                frame = buffer.tobytes()
                yield (b'--frame\r\nContent-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

        except Exception as err:
            print(err)
            tello_video.release()
        
            # sock.sendto("ccw 22.5".encode(), tello_address)
            # response = sock.recv(1024).decode()
            # print("response #4", response)
            # time.sleep(0.5)



@app.route('/video_feed_0')
def video_feed_0():
    return Response(gen_frames_camera(0), mimetype='multipart/x-mixed-replace; boundary=frame')


@app.route('/video_feed_1')
def video_feed_1():
    return Response(gen_frames_camera(2), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed_2')
def video_feed_2():
    return Response(gen_frames_camera(6), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed_drone')
def video_feed_drone():
    return Response(gen_frames_drone(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return flask.render_template('./index.html')

@app.route('/notification')
def notification():
    return flask.render_template('./notification.html', data=notifications)


app.run()

 * Serving Flask app 'hello'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Sep/2022 10:09:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2022 10:09:45] "GET /static/js/index.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 10:09:45] "GET /static/css/index.css HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 10:09:45] "GET /static/images/Ellipse_15.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 10:09:45] "GET /static/images/Ellipse_12.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 10:09:45] "GET /static/images/Ellipse_1.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 10:09:46] "GET /static/images/Vector_20.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 10:09:46] "GET /static/images/A_eye.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 10:09:46] "GET /static/images/v20_1259.png HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2022 10:09:46] "GET /static/images/v20_1234.png HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2022 10:09:46] "GET /static/images/1.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [18/Sep/2022 10:09:46] "GET

people talking 0.24070045351982117
people talking 0.23856619000434875
people talking 0.23520880937576294
people talking 0.23783203959465027
people talking 0.2343609780073166
Unknown 0.22791267931461334
Unknown 0.22751301527023315
Unknown 0.2278255671262741
Unknown 0.228629931807518
Unknown 0.2262425720691681
Unknown 0.2251097559928894
people talking 0.23270827531814575
Unknown 0.22393368184566498
Unknown 0.22469039261341095
people talking 0.23071128129959106
people talking 0.23506440222263336
people talking 0.24032826721668243
people talking 0.23033063113689423
Unknown 0.22618620097637177
Unknown 0.22111786901950836
Unknown 0.2297981083393097
people talking 0.23778776824474335
Unknown 0.22985012829303741
Unknown 0.21940691769123077
Unknown 0.22437500953674316
Unknown 0.2194732427597046
Unknown 0.2121661752462387
Unknown 0.21220901608467102
Unknown 0.20982863008975983
people talking 0.2322092354297638
Unknown 0.2262500524520874
people talking 0.23116165399551392
people talking 0.2313028

In [ ]:
# import the opencv library
import cv2
  
  
# define a video capture object
vid = cv2.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    label, conf, frame = process_stream(frame)
    
    # Display the resulting frame
    cv2.imshow("feed", frame)
    print(label, conf)
      
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

group of people 0.25326019525527954
person walking in office 0.25541919469833374
person walking in office 0.2482498288154602
person walking in office 0.25049325823783875
group of people 0.24347011744976044
person walking in office 0.24844025075435638
group of people 0.24437177181243896
person walking in office 0.2493806928396225
group of people 0.255823016166687
person walking in office 0.257604718208313
person walking in office 0.25367116928100586
person walking in office 0.24884331226348877
group of people 0.24825996160507202
group of people 0.24608902633190155
person walking in office 0.2490164190530777
person walking in office 0.2525266110897064
group of people 0.24728849530220032
group of people 0.24153855443000793
person walking in office 0.24188698828220367
person walking in office 0.240817129611969
person walking in office 0.24246889352798462
group of people 0.24239681661128998
person walking in office 0.24386774003505707
person walking in office 0.24390213191509247
group of pe

KeyboardInterrupt: 